At first we load the data and do the pre-processing on it 
1. check if there is any NULL in the dataset (Because it is the MNIST dataset and it is popular we did not check this part)
2. normalize the dataset with z-score and transfer the dataset to the new space with the mean of zero and the standard deviation of 1
(Normalizing is not necessary in this project because all the features are in the same range from 0 to 255 but we do not do it here because it cause a problem in the fiting of the autoencoder and it needs integers)

In [2]:
import pandas as pd
from scipy.stats import zscore

# Load the Data Set
data_labeled_train = pd.read_csv('labeled_train_set.csv')
data_labeled_test = pd.read_csv('test_set.csv')
data_unlabeled_train = pd.read_csv('unlabeled_train_set.csv')

# Conver the Data Set to Numpy Array and Spliting Labels and Features
X_train_labeled = data_labeled_train.iloc[: , 1:].values
y_train_labeled = data_labeled_train.iloc[: , 0].values
X_test_labeled = data_labeled_test.iloc[: , 1:].values
y_test_labeled = data_labeled_test.iloc[: , 0].values
X_train_unlabeled = data_unlabeled_train.iloc[:, :].values


Now we build an Auto Encoder and use it for Feature Selection and by that Reduce the dimentionality of the data
We are going to implement it in 5 layers: 
1. Input Layer
2. 3 Hidden Layers (128 , 16, 128)
3. Output Layer
So now it is obvious how the architecture of our Encoder and also Decoder is

In [3]:
from tensorflow import keras
from tensorflow.keras import layers

# Implimenting the Auto Encoder with the architecture we talked about

# Defining the Input Image dimention
input_image = keras.Input(shape=(784,))

# Define the Encoder
encoded = layers.Dense(128, activation='relu')(input_image)
encoded = layers.Dense(16, activation='relu')(encoded)

# Build the Encoder
encoder = keras.Model(input_image, encoded)

# Define the Decoder 
encoded_input = keras.Input(shape=(16,))
decoded = layers.Dense(128, activation='relu')(encoded_input)
decoded = layers.Dense(784, activation='sigmoid')(decoded)

# Build the Decoder 
decoder = keras.Model(encoded_input, decoded)

# Build the Auto Encoder
auto_encoder = keras.Model(input_image, decoder(encoder(input_image)))

# Compile the Auto Encoder
auto_encoder.compile(optimizer="adam", loss="binary_crossentropy")

# Fit the Data on our Auto Encoder
log = auto_encoder.fit(X_train_labeled, X_train_labeled, epochs=50, batch_size=256,
shuffle=True, validation_data=(X_test_labeled, X_test_labeled))
    

Epoch 1/50
71/71 [==============================] - 4s 18ms/step - loss: -1308892.3750 - val_loss: -8596151.0000
Epoch 2/50
71/71 [==============================] - 1s 15ms/step - loss: -90018432.0000 - val_loss: -298728128.0000
Epoch 3/50
71/71 [==============================] - 1s 14ms/step - loss: -997779712.0000 - val_loss: -2302154496.0000
Epoch 4/50
71/71 [==============================] - 1s 21ms/step - loss: -4949274624.0000 - val_loss: -9228468224.0000
Epoch 5/50
71/71 [==============================] - 1s 17ms/step - loss: -15920336896.0000 - val_loss: -25988585472.0000
Epoch 6/50
71/71 [==============================] - 1s 14ms/step - loss: -39345938432.0000 - val_loss: -58876444672.0000
Epoch 7/50
71/71 [==============================] - 1s 16ms/step - loss: -81976680448.0000 - val_loss: -115302137856.0000
Epoch 8/50
71/71 [==============================] - 1s 20ms/step - loss: -151399907328.0000 - val_loss: -203593842688.0000
Epoch 9/50
71/71 [=============================

After Fitting of the Auto Encoder is done, we encode the data set with it (reduce its dimentionality to 16)
P.S. I decide to reduce the dimentionality of the unlabeled data and then implement clustring algorithm on it to decrease the time of the running time

In [4]:
encoded_X_train_labeled = encoder.predict(X_train_labeled)
encoded_X_test_labeled = encoder.predict(X_test_labeled)
encoded_X_train_unlabeled = encoder.predict(X_train_unlabeled)


1313/1313 [==============================] - 3s 2ms/step


Before building the MLP model, I want to cluster the unlabeled data
Clustring Algorithm : K-Means

In [7]:
from sklearn.cluster import KMeans
import numpy as np

# Build the Model
k_means = KMeans(n_clusters=10)

# Fit Model on the Unlabeled Data
k_means.fit(encoded_X_train_unlabeled)

# Get the Labels
labels = np.array(k_means.labels_)


e:\tuts\DataMining\Homeworks\HW3\AutoEncoder\env\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[8 4 4 ... 3 6 7]
42000


Post-processing 
After the clustering we have to know that which lable correspond to which number from 0 to 9,
So I decide to find 1 instance from all of the labels and find their index in the unlabeled dataset that is not encoded because i've saved it for now, I could give their 16 features to decoder that I have built previously but I have the previous version of the dataset so I don't do that and at teh end visually Determine this thing

In [11]:
# Get instance from all of the labels
instances = []
index = 0
for i in range(len(labels)):
    if labels[i] == index:
        instances.append(i)
        index += 1
print(instances)

# Get their features
instances_features = []
for i in range(len(instances)):
    instances_features.append(X_train_unlabeled[instances[i]])
print(instances_features[0])
print(len(instances_features))

[11, 13, 14, 16, 36, 108, 113, 114, 118, 129]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0  16 166 253 253 255
 253 253 253 192 141 141 141  79  22   0   0   0   0   0   0   0   0   0
   0   0   0   0   0  29 252 252 252 253 252 252 252 253 252 252 252 253
 234  38   0   0   0   0   0   0   0   0   0   0   0   0   0  19 224 252
 252 253 252 252 252 253 252 252 252 253 252  94   0   0   0   0   0   0
   0   0   0   0   0   0  51 214 156 252 252 140 139 139 190 253 252 252
 252 253 177  94   0   0   0   0   0   0   0   0   0   0   0  48 241 255
 253 

With Pillow Library, I convert the matrixes to images

In [ ]:
from PIL import Image
